<a href="https://colab.research.google.com/github/ketannehra9/Predicitve-Analysis-Assignment5/blob/main/notebook/Copy_of_PAS_Assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# TOPSIS to find best pre-trained model for Text Generation
# ============================================================

!pip install -q torch transformers evaluate rouge_score sacrebleu nltk pandas matplotlib seaborn

import time
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import evaluate
import warnings
warnings.filterwarnings("ignore")

from transformers import AutoModelForCausalLM, AutoTokenizer

# dataset (generated by chatgpt)
data = [
    {"prompt": "The city experienced severe traffic congestion because",
     "reference": "multiple roads were closed for construction work."},

    {"prompt": "Remote work became popular during the pandemic since",
     "reference": "employees were required to work from home."},

    {"prompt": "Cybersecurity threats continue to rise as",
     "reference": "more systems become connected to the internet."},

    {"prompt": "A strong team performs well when",
     "reference": "members communicate openly and collaborate effectively."},

    {"prompt": "Online education platforms are useful because",
     "reference": "they allow students to learn from anywhere."},

    {"prompt": "Economic inflation affects consumers when",
     "reference": "prices increase faster than income."},

    {"prompt": "Climate change mitigation is difficult because",
     "reference": "it requires cooperation across multiple countries."},

    {"prompt": "Artificial intelligence is being adopted rapidly as",
     "reference": "businesses seek automation and efficiency."},

    {"prompt": "Data privacy concerns arise when",
     "reference": "personal information is collected without consent."},

    {"prompt": "Social media influences public opinion by",
     "reference": "spreading information quickly to large audiences."},

    {"prompt": "Electric vehicles are gaining popularity because",
     "reference": "they reduce dependence on fossil fuels."},

    {"prompt": "A good user interface improves usability by",
     "reference": "making applications easier to navigate."},

    {"prompt": "Cloud services are preferred since",
     "reference": "they provide scalable computing resources."},

    {"prompt": "Digital payments are increasing as",
     "reference": "people prefer cashless transactions."},

    {"prompt": "Good leadership is important because",
     "reference": "it guides teams toward shared goals."},

    {"prompt": "E-commerce platforms succeed when",
     "reference": "they offer convenience and reliability."},

    {"prompt": "System failures can be costly if",
     "reference": "critical services are interrupted."},

    {"prompt": "Mental health awareness has increased as",
     "reference": "society recognizes emotional well-being."},

    {"prompt": "Renewable energy adoption grows when",
     "reference": "environmental concerns become urgent."},

    {"prompt": "Problem solving skills improve when",
     "reference": "individuals practice logical thinking."},

    {"prompt": "Network latency becomes noticeable if",
     "reference": "data transmission is delayed."},

    {"prompt": "Security updates are essential because",
     "reference": "they protect systems from vulnerabilities."},

    {"prompt": "Team productivity increases when",
     "reference": "roles and responsibilities are clear."},

    {"prompt": "Mobile applications succeed when",
     "reference": "they deliver a smooth user experience."},

    {"prompt": "Technological innovation accelerates when",
     "reference": "research and development are well funded."}
]

# models compared
model_names = [
    "gpt2",
    "distilgpt2",
    "facebook/opt-125m",
    "EleutherAI/gpt-neo-125m",
    "microsoft/DialoGPT-small"
]

# parameters for topsis (asked from chatgpt)
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

param_cols = [
    "BLEU",
    "ROUGE-L",
    "METEOR",
    "Perplexity",
    "Inference Time (s)"
]

impacts = ["+", "+", "+", "-", "-"]
weights = [0.2] * 5

# evaluation
granular_data = []

for model_id in model_names:
    print(f"Processing model: {model_id}")
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    for i, item in enumerate(data):
        inputs = tokenizer(item["prompt"], return_tensors="pt")

        start_time = time.time()
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=20,
                do_sample=False,
                pad_token_id=tokenizer.pad_token_id
            )
        inf_time = time.time() - start_time

        gen_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        gen_text = gen_text.replace(item["prompt"], "").strip()

        b_score = bleu.compute(predictions=[gen_text], references=[item["reference"]])["bleu"]
        r_score = rouge.compute(predictions=[gen_text], references=[item["reference"]])["rougeL"]
        m_score = meteor.compute(predictions=[gen_text], references=[item["reference"]])["meteor"]

        ref_inputs = tokenizer(item["reference"], return_tensors="pt")
        with torch.no_grad():
            loss = model(ref_inputs["input_ids"], labels=ref_inputs["input_ids"]).loss
        p_score = torch.exp(loss).item()

        granular_data.append({
            "Prompt_ID": i + 1,
            "Model": model_id,
            "BLEU": b_score,
            "ROUGE-L": r_score,
            "METEOR": m_score,
            "Perplexity": p_score,
            "Inference Time (s)": inf_time
        })

df_granular = pd.DataFrame(granular_data)

# topsis function
def run_topsis(df_input, cols, impacts, weights):
    df_norm = df_input.copy()

    for i, col in enumerate(cols):
        denom = np.sqrt((df_input[col] ** 2).sum())

        if denom == 0:
            df_norm[col] = 0
        else:
            df_norm[col] = (df_input[col] / denom) * weights[i]

    ideal_best = []
    ideal_worst = []

    for i, col in enumerate(cols):
        if impacts[i] == "+":
            ideal_best.append(df_norm[col].max())
            ideal_worst.append(df_norm[col].min())
        else:
            ideal_best.append(df_norm[col].min())
            ideal_worst.append(df_norm[col].max())

    S_plus = []
    S_minus = []

    for _, row in df_norm.iterrows():
        d_pos = 0
        d_neg = 0
        for i, col in enumerate(cols):
            d_pos += (row[col] - ideal_best[i]) ** 2
            d_neg += (row[col] - ideal_worst[i]) ** 2
        S_plus.append(np.sqrt(d_pos))
        S_minus.append(np.sqrt(d_neg))

    S_plus = np.array(S_plus)
    S_minus = np.array(S_minus)

    return S_minus / (S_plus + S_minus + 1e-9)

# graph 1 - prompt-wise best model
winners = []

for pid in df_granular["Prompt_ID"].unique():
    slice_df = df_granular[df_granular["Prompt_ID"] == pid].copy()
    slice_df[param_cols] = slice_df[param_cols].fillna(0)
    slice_df["Score"] = run_topsis(slice_df, param_cols, impacts, weights)
    best_row = slice_df.loc[slice_df["Score"].idxmax()]
    winners.append({"Prompt_ID": pid, "Best Model": best_row["Model"]})

df_winners = pd.DataFrame(winners)

plt.figure(figsize=(10,6))
model_map = {m: i for i, m in enumerate(model_names)}
plt.scatter(
    df_winners["Prompt_ID"],
    [model_map[m] for m in df_winners["Best Model"]],
    s=200,
    edgecolors="black"
)
plt.yticks(range(len(model_names)), model_names)
plt.xlabel("Prompt ID")
plt.ylabel("Best Model")
plt.title("Best Model for Each Prompt (Prompt-wise TOPSIS)")
plt.grid(True, linestyle="--", alpha=0.6)
plt.show()

# graph 2 - topsis ranking
df_avg = df_granular.groupby("Model")[param_cols].mean().reset_index()
df_avg[param_cols] = df_avg[param_cols].fillna(0)
df_avg["TOPSIS Score"] = run_topsis(df_avg, param_cols, impacts, weights)
df_avg = df_avg.sort_values("TOPSIS Score", ascending=False)

plt.figure(figsize=(10,6))
plt.bar(df_avg["Model"], df_avg["TOPSIS Score"], edgecolor="black")
plt.xlabel("Model")
plt.ylabel("TOPSIS Score")
plt.title("Overall Model Ranking using TOPSIS")
plt.xticks(rotation=45, ha="right")
plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.show()

# final topsis table
df_avg


In [ ]:
df_avg = df_avg.sort_values(by="TOPSIS Score", ascending=False)
df_avg["Rank"] = range(1, len(df_avg) + 1)
df_avg = df_avg.reset_index(drop=True)

df_avg